In [ ]:
from IPython.utils import capture
with capture.capture_output() as cap:
  #
  from google.colab import drive
  drive.mount('/content/drive')
  #
  %cd /content
  #MyDrive/dataset: inside add your folder with images, example: 1_onx woman
  !mkdir /content/drive/MyDrive/dataset
  #MyDrive/train: here the lora and states are stored
  !mkdir /content/drive/MyDrive/train
  !mkdir /content/log
  !mkdir -p /root/.cache/huggingface/accelerate/
  #
  #git clone https://github.com/kohya-ss/sd-scripts --branch sd3
  !curl -Lo /content/zeroLR.zip https://huggingface.co/josemerinom/flux/resolve/main/zeroLR_sd3_feb26.zip
  #accelarator config (bp16)
  !curl -Lo /content/default_config.yaml https://huggingface.co/josemerinom/flux/resolve/main/default_config_33.yaml
  #vae
  !curl -Lo /content/ae.safetensors https://huggingface.co/josemerinom/flux/resolve/main/ae.safetensors
  #clip
  !curl -Lo /content/clip_l.safetensors https://huggingface.co/josemerinom/flux/resolve/main/clip_l.safetensors
  #text FP8
  !curl -Lo /content/t5xxl_fp8_e4m3fn.safetensors https://huggingface.co/josemerinom/flux/resolve/main/t5xxl_fp8_e4m3fn.safetensors
  #unet FP8
  !curl -Lo /content/flux1-dev-fp8.safetensors https://huggingface.co/josemerinom/flux/resolve/main/flux1-dev-fp8.safetensors
  #
  !unzip -u /content/zeroLR.zip
  !cp -r /content/default_config.yaml /root/.cache/huggingface/accelerate/
  #
  #kohya folder (I renamed it to zeroLR)
  %cd /content/zeroLR
  #
  #Uninstalling libraries to avoid conflicts
  !pip uninstall -y peft
  !pip uninstall -y accelerate
  !pip uninstall -y torch torchvision torchaudio
  #
  #installing libraries manually (there is the option to use the req..txt file)
  !pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124
  !pip install accelerate==0.33.0
  !pip install transformers==4.44.0
  !pip install diffusers[torch]==0.25.0
  !pip install ftfy==6.1.1
  !pip install albumentations==1.3.0
  !pip install opencv-python==4.8.1.78
  !pip install einops==0.7.0
  !pip install pytorch-lightning==1.9.0
  !pip install bitsandbytes==0.44.0
  !pip install prodigyopt==1.0
  !pip install lion-pytorch==0.0.6
  !pip install schedulefree==1.4
  !pip install tensorboard
  !pip install safetensors==0.4.4
  !pip install altair==4.2.2
  !pip install easygui==0.98.3
  !pip install toml==0.10.2
  !pip install voluptuous==0.13.1
  !pip install huggingface-hub==0.24.5
  !pip install imagesize==1.4.1
  !pip install numpy<=2.0
  !pip install rich==13.7.1
  !pip install sentencepiece==0.2.0
  !pip install -e .
  #
print("done")

In [ ]:
#The following configuration is made to use 14.7gb / 15.0gb vram
#The text_encoder is not trained (it can be activated, delete: --network_train_unet_only \)
#
#If you change a parameter and the vram consumption increases:
#Change the value of --blocks_to_swap to a higher one
#
#--output_name here goes the name that your training/files will take
#
#To continue training add the parameter:
#--resume and indicate the last stored state. example: --resume="/content/drive/MyDrive/train/...state..."

In [ ]:
%cd /content/zeroLR
#
!accelerate launch --num_processes=1 --num_machines=1 --num_cpu_threads_per_process=1 flux_train_network.py \
  --output_name="test_v1" \
  --adaptive_noise_scale=0 \
  --ae="/content/ae.safetensors" \
  --blocks_to_swap=12 \
  --caption_extension=".txt" \
  --cache_latents \
  --clip_l="/content/clip_l.safetensors" \
  --clip_skip=1 \
  --console_log_simple \
  --discrete_flow_shift=3.1582 \
  --fp8_base \
  --full_bf16 \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --guidance_scale=1 \
  --highvram \
  --huber_c=0.1 \
  --huber_schedule="snr" \
  --ip_noise_gamma=0 \
  --keep_tokens=1 \
  --learning_rate=4e-4 \
  --logging_dir="/content/log" \
  --loss_type="l2" \
  --lr_scheduler="constant" \
  --lr_scheduler_num_cycles=1 \
  --lr_scheduler_power=1 \
  --lr_warmup_steps=0 \
  --max_data_loader_n_workers=2 \
  --max_train_epochs=70 \
  --min_snr_gamma=0 \
  --mixed_precision="bf16" \
  --model_prediction_type="raw" \
  --network_alpha=8 \
  --network_dim=8 \
  --network_dropout=0 \
  --network_module=networks.lora_flux \
  --network_train_unet_only \
  --noise_offset=0 \
  --output_dir="/content/drive/MyDrive/train" \
  --persistent_data_loader_workers \
  --pretrained_model_name_or_path="/content/flux1-dev-fp8.safetensors" \
  --prior_loss_weight=1 \
  --resolution=512 \
  --save_every_n_epochs=1 \
  --save_model_as="safetensors" \
  --save_precision="bf16" \
  --save_state \
  --scale_weight_norms=0 \
  --sdpa \
  --seed=42 \
  --sigmoid_scale=1 \
  --t5xxl="/content/t5xxl_fp8_e4m3fn.safetensors" \
  --text_encoder_lr=0 \
  --timestep_sampling="shift" \
  --train_batch_size=1 \
  --train_data_dir="/content/drive/MyDrive/dataset" \
  --unet_lr=4e-4 \
  --optimizer_args "betas=(0.9, 0.999)" "eps=1e-8" "weight_decay=0.01" \
  --optimizer_type="adamw8bit" \
  --max_grad_norm=1 \
#

In [ ]:
#ADAFACTOR
#--optimizer_args "scale_parameter=False" "relative_step=False" "warmup_init=False" \
#--optimizer_type="Adafactor" \
#--max_grad_norm=0 \
#
#ADAMW8BIT
#--optimizer_args "betas=(0.9, 0.999)" "eps=1e-8" "weight_decay=0.01" \
#--optimizer_type="adamw8bit" \
#--max_grad_norm=1 \
#
#CONTINUE TRAINING
#--resume="/content/drive/MyDrive/train/...."
#
#IF YOU DO NOT WANT TO SAVE FOR EVERY ERA, YOU CAN CHANGE THE SAVE BY STEP
#--save_every_n_steps=100
#
#OTHER PARAMETERS
#--max_train_steps=1400 \
#--cache_text_encoder_outputs \
#--cache_text_encoder_outputs_to_disk \
#--network_args "train_t5xxl=True" \
#--apply_t5_attn_mask \

In [ ]:
#CLOSE COLAB SESSION AFTER 1 MIN
from google.colab import runtime
import time
print('Done')
time.sleep(60)
runtime.unassign()